# CVAE

The encoder and decoder in addition to the input data are provided with an embedding vector that represents a condition. Thus, the encoder does not need to represent the condition in the latent space since the decoder will also get this information as an extra input.

In this example we will use the MNIST dataset: the encoder can regress out the condition (specific digits) and learn the handwriting style as a latent representation.

In [ ]:
# Install the library

!pip install rapidae

In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

notebook_dir = os.path.abspath('') # get the current notebook directory
sys.path.append(os.path.join(notebook_dir, 'rapidae', 'src')) # add src folder to path to import modules
                                                        # '..', 'src' if you are in the 'examples' folder

In [22]:
from rapidae.data import load_dataset
from rapidae.models import CVAE
from rapidae.models.base import VAE_Encoder_MLP, VAE_Decoder_MLP
from rapidae.pipelines import TrainingPipeline
from rapidae.evaluate import plot_latent_space, plot_reconstructions

### Data

In [20]:
# Load MNIST dataset
data = load_dataset("MNIST")

# normalize data
x_train = data["x_train"].reshape(data["x_train"].shape[0], -1) / 255
x_test = data["x_test"].reshape(data["x_test"].shape[0], -1) / 255
y_train = data["y_train"]
y_test = data["y_test"]

print("Data shape:", x_train.shape)

2024-05-06 18:10:32 [INFO]: Downloading data...


Data shape: (60000, 784)


Keep only images of 0–7 digits

In [11]:
import numpy as np

idx_07 = np.where((data["y_train"] != 8) & (data["y_train"] == 9))[0]
x_train_07 = x_train[idx_07]
y_train_07 = y_train[idx_07]

### Model

In [23]:
input_dim = x_train.shape[1:]
# Model creation
model = CVAE(input_dim=input_dim, 
            latent_dim=2,
            encoder=VAE_Encoder_MLP(input_dim=input_dim, latent_dim=2), 
            decoder=VAE_Decoder_MLP(input_dim=input_dim, latent_dim=2))

2024-05-06 18:12:44 [INFO]: Using provided encoder
2024-05-06 18:12:44 [INFO]: Using provided decoder


### Training

In [24]:
pipe = TrainingPipeline(name='CVAE_MNIST', 
                        learning_rate=0.001,
                        model=model, 
                        num_epochs=30, 
                        batch_size=128,
                        run_eagerly=True,)

trained_model = pipe(x=(x_train, y_train))

2024-05-06 18:12:47 [INFO]: +++ CVAE_MNIST +++
2024-05-06 18:12:47 [INFO]: Creating folder in ./output_dir/CVAE_MNIST_2024-05-06_18-12
2024-05-06 18:12:47 [INFO]: 
TRAINING STARTED
	Backend: tensorflow
	Eager mode: True
	Validation data available: False
	Callbacks set: ['EarlyStopping', 'ModelCheckpoint'] 



2024-05-06 18:12:47.145814: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 188160000 exceeds 10% of free system memory.


Epoch 1/30


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/keras/src/layers/layer.py:1265: UserWarning: Layer 'cvae_1' looks like it has unbuilt state, but Keras is not able to trace the layer `call()` in order to build it automatically. Possible causes:
1. The `call()` method of your layer may be crashing. Try to `__call__()` the layer eagerly on some test input first to see if it works. E.g. `x = np.random.random((3, 4)); y = layer(x)`
2. If the `call()` method is correct, then you may need to implement the `def build(self, input_shape)` method on your layer. It should create all variables used by the layer (e.g. by calling `layer.build()` on all its children layers).
Exception encountered: ''Exception encountered when calling VAE_Decoder_MLP.call().

Invalid dtype: tuple

Arguments received by VAE_Decoder_MLP.call():
  • z=tf.Tensor(shape=(128, 4), dtype=float32)''
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/keras/src/layers/layer.py:3

ValueError: Exception encountered when calling VAE_Decoder_MLP.call().

[1mInvalid dtype: tuple[0m

Arguments received by VAE_Decoder_MLP.call():
  • z=tf.Tensor(shape=(128, 4), dtype=float32)

Aparentemente z es una tupla, averiguar por qué